<a href="https://colab.research.google.com/github/mikecinnamon/MLearning/blob/main/Notebooks/fake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ML-23] Example - Fake news detection

## Importing the data

In [ ]:
import pandas as pd, numpy as np
path = 'https://raw.githubusercontent.com/mikecinnamon/Data/main/'
df1 = pd.read_csv(path + 'fake1.csv.zip', index_col=0)
df2 = pd.read_csv(path + 'fake2.csv.zip', index_col=0)
df = pd.concat([df1, df2])

In [ ]:
df.info()

In [ ]:
df.head()

## Q1. Cleaning the data

In [ ]:
df = df.drop(columns=['author']).dropna()
df.info()

In [ ]:
pd.concat([df['title'].str.len().describe(), df['text'].str.len().describe()], axis=1)

In [ ]:
df['title'][df['title'].str.len() < 5]

In [ ]:
df['label'].mean().round(3)

## Q2. Encoding the titles

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('all-MiniLM-L12-v2')

In [ ]:
title = df['title'].to_list()

In [ ]:
title_embed = model.encode(title, output_value='sentence_embedding')

In [ ]:
title_embed.shape

## Q3. Logistic regression model

In [ ]:
y = df['label']
X = title_embed

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

In [ ]:
clf.fit(X, y)
round(clf.score(X, y), 3)

In [ ]:
y_pred = clf.predict(X)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_pred)

## Q4. Distribution of the predictive scores

In [ ]:
df['title_score'] = clf.predict_proba(X)[:, 1]

In [ ]:
from matplotlib import pyplot as plt
# Set the size of the figure
plt.figure(figsize=(12,5))
# First subplot
plt.subplot(1, 2, 1)
plt.hist(df['title_score'][y == 1], range=(0,1), color='gray', edgecolor='white')
plt.title('Figure 1.a. Scores (fakes)')
plt.xlabel('Fake score')
# Second subplot
plt.subplot(1, 2, 2)
plt.hist(df['title_score'][y == 0], range=(0,1), color='gray', edgecolor='white')
plt.title('Figure 1.b. Scores (non-fakes)')
plt.xlabel('Fake score');